# **Predictive Future Sales**
**1. Exploratory Data Analysis**

**Introduction:**
In this competition you are provided with a challenging time-series dataset consisting of daily sales data, kindly provided by one of the largest Russian software firms - 1C Company. We are asking you to predict total sales for every product and store in the next month. The task is to forecast the total amount of products sold in every shop for the test set. Note that the list of shops and products changes every month. 
This notebook contains general information about the methods using which you can approach the problem statement.

Lets first discuss what we are given and what we have to predict. About our dataset:

The features in our *training* data:

1. date - every date of items sold
2. date_block_num - this number is given to every month
3. shop_id - unique number of every shop
4. item_id - unique number of every item
5. item_price - price of every item
6. item_cnt_day - number of items sold on a particular day

The features in our *testing* data :

1. ID - unique for every (shop_id,item_id) pair.
2. shop_id - unique number of every shop
3. item_id - unique number of every item

Daily historical sales are given *from Jan 2013 to Oct 2015*. The **task** is to predict total sales for every product and store in the next month. The **goal** is here to minimise the performance metric:*RMSE score*.

In [3]:
#import python packages and libs#
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import datetime
import warnings
from xgboost import XGBRegressor
from xgboost import plot_importance
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler

In [4]:
#import the raw datasets#
train=pd.read_csv("../input/competitive-data-science-predict-future-sales/sales_train.csv")
items=pd.read_csv("../input/competitive-data-science-predict-future-sales/items.csv")
categories=pd.read_csv('../input/competitive-data-science-predict-future-sales/item_categories.csv')
shops=pd.read_csv('../input/competitive-data-science-predict-future-sales/shops.csv')
test=pd.read_csv('../input/competitive-data-science-predict-future-sales/test.csv')

**Dataset Overview**

Datasets apart from the test and train dataset that are given to us:

1. item_categories.csv - the item category name along with the category ID
2. items.csv - the item name along with item ID and category ID
3. shops.csv - the shop name along with shop ID

**1.1Categories**

In [5]:
print("First 5 Entries")
print(categories.head(5))

print("Information")
print(categories.info())

print("Data Types")
print(categories.dtypes)

print("Missing Value")
print(categories.isnull().sum())

print("Null Value")
print(categories.isna().sum())

print("Shape")
print(categories.shape)

print('Description')
categories.describe()

**1.2 Items**

In [6]:
print("First 5 Entries")
print(items.head(5))

print("Information")
print(items.info())

print("Data Types")
print(items.dtypes)

print("Missing value")
print(items.isnull().sum())

print("Null value")
print(items.isna().sum())

print("Shape of Data")
print(items.shape)

print('Description')
items.describe()

**1.3 Shops**

In [7]:
print("First 5 Entries")
print(shops.head(5))

print("Info")
print(shops.info())

print("Data Types")
print(shops.dtypes)

print("Missing value")
print(shops.isnull().sum())

print("Null value")
print(shops.isna().sum())

print("Shape of Data")
print(shops.shape)

print('Description')
shops.describe()

**1.4 Training**

In [8]:
print("First 5 entries")
print(train.head())

In [9]:
print("Shape")
print(train.shape)

In [10]:
print("Infomation")
print(train.info())

In [11]:
print("Data Types")
print(train.dtypes)

In [12]:
print("Missing NaN values")
print(train.isnull().sum())

In [13]:
print("Null Values")
print(train.isna().sum())

In [14]:
train.describe()

In [15]:
#Change the datetime format
train['date'] = pd.to_datetime(train['date'], format = '%d.%m.%Y')
train

In [16]:
#join the training dataset
train = train.join(items, on='item_id', rsuffix='_').join(shops, on='shop_id', rsuffix='_').join(categories, on='item_category_id', rsuffix='_')

In [17]:
print('Min date from train set: %s' % train['date'].min().date())
print('Max date from train set: %s' % train['date'].max().date())

In [18]:
#Note that this function can be modified by choosing to calculate duplicates over only a subset of features, keeping the last entry, etc.
train.drop_duplicates(inplace=True,keep='first')
print(train)

In [19]:
#Data cleaning and remove rows with negative item price.
train = train.query('item_price > 0') 
train

**Data Leakage🌟**

When there's data leakage in the data used for machine learning model,we will get a high train and test accuracy, implying that the model is good enough for production. It will neither underfit or overfit.

However, when implementing the machine learning model in production, it will no longer be introduced to one feature. Because it is not available when you need the model’s predictions. The feature missing might even be the most important feature for determining the right class: the leaked data.

When implying the machine learning model in production, we will see that the predictions are not reliable.

We'll only be using only the "shop_id" and "item_id" that appear on the test dataset. The idea here is that we know on which shops and items we are going to predict, because of the test set, training on those rows you will get a data distribution closer to the test set, so probably this is a better idea.

In [20]:
test_shop_ids = test['shop_id'].unique()
test_item_ids = test['item_id'].unique()
# Only shops that exist in test set.
lk_train = train[train['shop_id'].isin(test_shop_ids)]
# Only items that exist in test set.
lk_train = lk_train[lk_train['item_id'].isin(test_item_ids)]

print('Data set size before leaking:', train.shape[0])
print('Data set size after leaking:', lk_train.shape[0])

In [21]:
#After leakage,drop the text columns since they are not significant for predictions.
lk_train

In [22]:
lk_train.drop(['item_name','shop_name','item_category_name'],axis=1)

In [23]:
train_monthly = lk_train[['date', 'date_block_num', 'shop_id', 'item_id', 'item_price', 'item_cnt_day','item_category_id']]
train_monthly.head()

**1.5 EDA for Item_id**
1. How the monthly sum and mean vary for with item_id for each month
2. The outdated items, over perhaps the last 6 months.

In [24]:
train_by_item_id = train.pivot_table(index=['item_id'],values=['item_cnt_day'], columns='date_block_num', aggfunc=np.sum, fill_value=0).reset_index()
train_by_item_id.columns = train_by_item_id.columns.droplevel().map(str)
train_by_item_id = train_by_item_id.reset_index(drop=True).rename_axis(None, axis=1)
train_by_item_id.columns.values[0] = 'item_id'
# print(train_by_item_id.sum()[1:])
train_by_item_id.sum()[1:].plot(legend=True, label="Monthly sum")

In [25]:
train_by_item_id.mean()[1:].plot(legend=True, label="Monthly mean")

In [26]:
train.item_id.nunique()

In [27]:
#outdated after month 27 in our 34 month window
outdated_items = train_by_item_id[train_by_item_id.loc[:,'27':].sum(axis=1)==0] 
print('Outdated items:', len(outdated_items))

In [28]:
print('Outdated items in test set:', len(test[test['item_id'].isin(outdated_items['item_id'])]))

**Summary**
1. There are no missing values at all. 
2. Number of sold items usually declines over the year.
3. There are peaks in December and similar item count zig-zag behavior can be seen in June-July-August. This can be due to these periods being vacation time or possibly a national holiday.
4. 12391 of 21807 items in the train set are outdated since the last 6 months, which is quite huge.
5. For test set: 6888 outdated.

**1.6 EDA for Shop_id**

In [29]:
train_by_shop_id = train.pivot_table(index=['shop_id'],values=['item_cnt_day'], columns='date_block_num', aggfunc=np.sum, fill_value=0).reset_index()
train_by_shop_id.columns = train_by_shop_id.columns.droplevel().map(str)
train_by_shop_id = train_by_shop_id.reset_index(drop=True).rename_axis(None, axis=1)
train_by_shop_id.columns.values[0] = 'shop_id'

for i in range(6,34):
    print('Does not exist in month',i,train_by_shop_id['shop_id'][train_by_shop_id.loc[:,'0':str(i)].sum(axis=1)==0].unique())

for i in range(6,28):
    print('Shop is outdated for month',i,train_by_shop_id['shop_id'][train_by_shop_id.loc[:,str(i):].sum(axis=1)==0].unique())

In [30]:
print('Recently opened shop items:', len(test[test['shop_id']==36])) 

**2.Data Preprocessing**

1. We drop irrelevant or less important features
2. We need to process datetime, from daily-->monthly

In [31]:
#Select useful features
train_monthly = lk_train[['date', 'date_block_num', 'shop_id', 'item_category_id', 'item_id', 'item_price', 'item_cnt_day']]
train_monthly.head()

In [32]:
# Group by month in this case "date_block_num" and aggregate features.
train_monthly = train_monthly.sort_values(['date']).groupby(['date_block_num', 'shop_id', 'item_category_id', 'item_id'], as_index=False)
train_monthly = train_monthly.agg({'item_price':['sum', 'mean'], 'item_cnt_day':['sum', 'mean','count']})

In [33]:
# Rename features.
train_monthly.columns = ['date_block_num', 'shop_id', 'item_category_id', 'item_id', 'item_price', 'mean_item_price', 'item_cnt', 'mean_item_cnt','transactions']

We're using empty_df, we need a dataframe that will have combinations of months, shop_id, and item_id.
First create an empty dataframe, then iterate over the existing records to fill it, and finally fill the missing values with 0.
We're taking all possible combinations here since we have to tell the model that for those months the item count for a particular shop ID/item ID was zero instead of having any missing records.

In [34]:
shop_ids = train_monthly['shop_id'].unique()
item_ids = train_monthly['item_id'].unique()
empty_df = []
for i in range(34):
    for shop in shop_ids:
        for item in item_ids:
            empty_df.append([i, shop, item])
empty_df = pd.DataFrame(empty_df, columns=['date_block_num','shop_id','item_id'])

In [35]:
#Merge the train set with the complete set (missing records will be filled with 0).
train_monthly = pd.merge(empty_df, train_monthly, on=['date_block_num','shop_id','item_id'], how='left')
train_monthly.fillna(0, inplace=True)

In [36]:
train_monthly.head()

In [37]:
train_monthly.describe()

In [38]:
# Extract time based features, this will be essential when we group the data based on month/year.
train_monthly['year'] = train_monthly['date_block_num'].apply(lambda x: ((x//12) + 2013))
train_monthly['month'] = train_monthly['date_block_num'].apply(lambda x: (x % 12))

In [39]:
#date_block_num covers a window of 34 months and takes values from 0 to 33, so to extract some 'monthly' features
gp_month_mean = train_monthly.groupby(['month'], as_index=False)['item_cnt'].mean()
gp_month_sum = train_monthly.groupby(['month'], as_index=False)['item_cnt'].sum()
gp_category_mean = train_monthly.groupby(['item_category_id'], as_index=False)['item_cnt'].mean()
gp_category_sum = train_monthly.groupby(['item_category_id'], as_index=False)['item_cnt'].sum()
gp_shop_mean = train_monthly.groupby(['shop_id'], as_index=False)['item_cnt'].mean()
gp_shop_sum = train_monthly.groupby(['shop_id'], as_index=False)['item_cnt'].sum()

In [40]:
plt.style.use('seaborn')
train.copy().set_index('date').item_cnt_day.resample('M').mean().plot()

In [41]:
f, axes = plt.subplots(2, 1, figsize=(22, 10), sharex=True)
sns.lineplot(x="month", y="item_cnt", data=gp_month_mean, ax=axes[0]).set_title("Monthly mean")
sns.lineplot(x="month", y="item_cnt", data=gp_month_sum, ax=axes[1]).set_title("Monthly sum")
plt.show()

**Summary**
1. We have a trending increase of item sales count (mean) towards the ending of the year.
2. There are peaks in October, then dips in November, followed by an increase in December and similar item count zig-zag behavior can be seen in June-July-August. This can be due to: vacation time/national holidays.

**Item Category Sales Viz**

In [42]:
fig, axes = plt.subplots(2, 1, figsize=(22, 10), sharex=True)
sns.barplot(x="item_category_id", y="item_cnt", data=gp_category_mean, ax=axes[0], palette="rocket").set_title("Monthly Mean")
sns.barplot(x="item_category_id", y="item_cnt", data=gp_category_sum, ax=axes[1], palette="rocket").set_title("Monthly Sum")
plt.show()

In conclusion,few of the categories are dominating the market.

**Shops Sales Viz**

In [43]:
fig, axes = plt.subplots(2, 1, figsize=(22, 10), sharex=True)
sns.barplot(x="shop_id", y="item_cnt", data=gp_shop_mean, ax=axes[0], palette="rocket").set_title("Monthly mean")
sns.barplot(x="shop_id", y="item_cnt", data=gp_shop_sum, ax=axes[1], palette="rocket").set_title("Monthly sum")
plt.show()

**Conclusion:** Most of the shops have a similar sell rate, but 3 of them have a much higher rate, because of their shop size.

**Outlier Identification**

In [44]:
plt.figure(figsize=(15,4))
plt.xlim(-100, 3000)
sns.boxplot(x=train['item_cnt_day'])
print('Item Sale outliers:',train['item_id'][train['item_cnt_day']>500].unique())

plt.figure(figsize=(15,4))
plt.xlim(train['item_price'].min(), train['item_price'].max())
sns.boxplot(x=train['item_price'])
print('Item price outliers:',train['item_id'][train['item_price']>50000].unique())

In [45]:
#drop the outliers
train_monthly = train_monthly.query('item_cnt >= 0 and item_cnt <= 500 and item_price < 50000')
train_monthly

**3.Feature Engineering**

In [46]:
#shift the time since it is a forecast problem
train_monthly['item_cnt_month'] = train_monthly.sort_values('date_block_num').groupby(['shop_id', 'item_id'])['item_cnt'].shift(-1) 
train_monthly

In [47]:
#unify the price mark
train_monthly['item_price_unit'] = train_monthly['item_price'] // train_monthly['item_cnt']
train_monthly['item_price_unit'].fillna(0, inplace=True)

In [48]:
#grouping data with item_id
gp_item_price = train_monthly.sort_values('date_block_num').groupby(['item_id'], as_index=False).agg({'item_price':[np.min, np.max]})
gp_item_price.columns = ['item_id', 'hist_min_item_price', 'hist_max_item_price']

train_monthly = pd.merge(train_monthly, gp_item_price, on='item_id', how='left')

In [49]:
train_monthly['price_increase'] = train_monthly['item_price'] - train_monthly['hist_min_item_price']
train_monthly['price_decrease'] = train_monthly['hist_max_item_price'] - train_monthly['item_price']

**Rolling Window**

1. Creating a rolling window with a specified size and perform calculations on the data
2. A time series problem that recent lag values are more predictive than older lag values
3. rolling() functions->perform rolling window functions.Regard window size as a parameter to group values.

In [50]:
#make rolling functions' parameters with lambda function
#Min value
f_min = lambda x: x.rolling(window=3, min_periods=1).min()
#Max value
f_max = lambda x: x.rolling(window=3, min_periods=1).max()
#Mean value
f_mean = lambda x: x.rolling(window=3, min_periods=1).mean()
#Standard deviation
f_std = lambda x: x.rolling(window=3, min_periods=1).std()

function_list = [f_min, f_max, f_mean, f_std]
function_name = ['min', 'max', 'mean', 'std']

for i in range(len(function_list)):
    train_monthly[('item_cnt_%s' % function_name[i])] = train_monthly.sort_values('date_block_num').groupby(['shop_id', 'item_category_id', 'item_id'])['item_cnt'].apply(function_list[i])
#Fill the empty std features with 0
train_monthly['item_cnt_std'].fillna(0, inplace=True)


**Lag Based Feature**
1. A lag features is a variable which contains data from **prior time steps**.
2. Lag features are a classical way for transforming a time series forecasting problem into a **supervised learning problem**.
3. Let's say you are predicting the stock price for a company. So, the previous day’s stock price is vital in making predictions right? So, this means that the value at time t is greatly affected by the value at time t-1. The past values are known as lags, so **t-1 is lag 1, t-2 is lag 2,** and so on.

In [51]:
#create the lag lists for lag1, lag2, lag3 with prior time data
lag_list = [1, 2, 3]
#name the lag list features
for lag in lag_list:
    ft_name = ('item_cnt_shifted%s' % lag)
    train_monthly[ft_name] = train_monthly.sort_values('date_block_num').groupby(['shop_id', 'item_category_id', 'item_id'])['item_cnt'].shift(lag)
    # Fill the empty shifted features with 0
    train_monthly[ft_name].fillna(0, inplace=True)

In [52]:
#generate monthly_trend variable
train_monthly['item_trend'] = train_monthly['item_cnt']

for lag in lag_list:
    ft_name = ('item_cnt_shifted%s' % lag)
    train_monthly['item_trend'] -= train_monthly[ft_name]

train_monthly['item_trend'] /= len(lag_list) + 1
train_monthly.head().T

We need to pre-define the train and test datasets for this time series data.

The test set is in the future, so we simulate the **same distribution** on our train/validation split.

Our **train set** will be the first 3-27 blocks, **validation** will be last 5 blocks (28-32) and **test** will be block 33.

(We can leave out the first 3 months because we use a 3 month window to generate features, so these first 3 month won't have really windowed useful features.)

In [53]:
#query the train,test, and validation sets with pre-defined blocks
train_set = train_monthly.query('date_block_num >= 3 and date_block_num < 28').copy()
validation_set = train_monthly.query('date_block_num >= 28 and date_block_num < 33').copy()
test_set = train_monthly.query('date_block_num == 33').copy()
#deal with missing data
train_set.dropna(subset=['item_cnt_month'], inplace=True)
validation_set.dropna(subset=['item_cnt_month'], inplace=True)

train_set.dropna(inplace=True)
validation_set.dropna(inplace=True)

print('Train set records:', train_set.shape[0])
print('Validation set records:', validation_set.shape[0])
print('Test set records:', test_set.shape[0])


**Mean encoding** can be quite helpful for the model which we use (a gradient boosting model), and represent the features in a better way. Since we have a lot of different values for shop_ids, item_ids, this can also help in reducing cardinality.

In [54]:
 # Shop mean encoding.
gp_shop_mean = train_set.groupby(['shop_id']).agg({'item_cnt_month': ['mean']})
gp_shop_mean.columns = ['shop_mean']
gp_shop_mean.reset_index(inplace=True)
# Item mean encoding.
gp_item_mean = train_set.groupby(['item_id']).agg({'item_cnt_month': ['mean']})
gp_item_mean.columns = ['item_mean']
gp_item_mean.reset_index(inplace=True)
# Shop with item mean encoding.
gp_shop_item_mean = train_set.groupby(['shop_id', 'item_id']).agg({'item_cnt_month': ['mean']})
gp_shop_item_mean.columns = ['shop_item_mean']
gp_shop_item_mean.reset_index(inplace=True)
# Year mean encoding.
gp_year_mean = train_set.groupby(['year']).agg({'item_cnt_month': ['mean']})
gp_year_mean.columns = ['year_mean']
gp_year_mean.reset_index(inplace=True)
# Month mean encoding.
gp_month_mean = train_set.groupby(['month']).agg({'item_cnt_month': ['mean']})
gp_month_mean.columns = ['month_mean']
gp_month_mean.reset_index(inplace=True)

In [55]:
# Add mean encoding features to train set.
train_set = pd.merge(train_set, gp_shop_mean, on=['shop_id'], how='left')
train_set = pd.merge(train_set, gp_item_mean, on=['item_id'], how='left')
train_set = pd.merge(train_set, gp_shop_item_mean, on=['shop_id', 'item_id'], how='left')
train_set = pd.merge(train_set, gp_year_mean, on=['year'], how='left')
train_set = pd.merge(train_set, gp_month_mean, on=['month'], how='left')
# Add mean encoding features to validation set.
validation_set = pd.merge(validation_set, gp_shop_mean, on=['shop_id'], how='left')
validation_set = pd.merge(validation_set, gp_item_mean, on=['item_id'], how='left')
validation_set = pd.merge(validation_set, gp_shop_item_mean, on=['shop_id', 'item_id'], how='left')
validation_set = pd.merge(validation_set, gp_year_mean, on=['year'], how='left')
validation_set = pd.merge(validation_set, gp_month_mean, on=['month'], how='left')

In [56]:
# Create train and validation sets and labels. 
X_train = train_set.drop(['item_cnt_month', 'date_block_num'], axis=1)
Y_train = train_set['item_cnt_month'].astype(int)
X_validation = validation_set.drop(['item_cnt_month', 'date_block_num'], axis=1)
Y_validation = validation_set['item_cnt_month'].astype(int)

In [57]:
int_features = ['shop_id', 'item_id', 'year', 'month']

X_train[int_features] = X_train[int_features].astype('int32')
X_validation[int_features] = X_validation[int_features].astype('int32')

In [58]:
latest_records = pd.concat([train_set, validation_set]).drop_duplicates(subset=['shop_id', 'item_id'], keep='last')
X_test = pd.merge(test, latest_records, on=['shop_id', 'item_id'], how='left', suffixes=['', '_'])
X_test['year'] = 2015
X_test['month'] = 9
X_test.drop('item_cnt_month', axis=1, inplace=True)

X_test = X_test[X_train.columns]

In [59]:
sets = [X_train, X_validation, X_test]
# Replace missing values with the median of each shop.            
for dataset in sets:
    for shop_id in dataset['shop_id'].unique():
        for column in dataset.columns:
            shop_median = dataset[(dataset['shop_id'] == shop_id)][column].median()
            dataset.loc[(dataset[column].isnull()) & (dataset['shop_id'] == shop_id), column] = shop_median
            
# Fill remaining missing values on test set with mean.
X_test.fillna(X_test.mean(), inplace=True)

In [60]:
# I'm dropping "item_category_id", we don't have it on test set and would be a little hard to create categories for items that exist only on test set.
X_train.drop(['item_category_id'], axis=1, inplace=True)
X_validation.drop(['item_category_id'], axis=1, inplace=True)
X_test.drop(['item_category_id'], axis=1, inplace=True)

In [61]:
X_test.head().T

**4.Modelling**

**4.1XGBoost**

A gradient boosting model works by adding predictors to an ensemble in a sequential fashion, with the new predictor being fit to the residual errors made by the previous predictor.

In [62]:
# Use part of features on XGBoost.
xgb_features = ['item_cnt','item_cnt_mean', 'item_cnt_std', 'item_cnt_shifted1', 'item_cnt_shifted2', 'item_cnt_shifted3', 'shop_mean', 'shop_item_mean', 'item_trend', 'mean_item_cnt']
xgb_train = X_train[xgb_features]
xgb_val = X_validation[xgb_features]
xgb_test = X_test[xgb_features]

In [63]:
#set up and fit the model
from xgboost import XGBRegressor
xgb_model = XGBRegressor(max_depth=10,  #maximum depth of the decision trees being trained,if this value increase, more complex is the model,more likely to be overfitted.default=6
                         n_estimators=500,  #500 trees are ensembled for efficient learning of data 
                         min_child_weight=1000,  #default=1.建立每节需要的样本数量
                         colsample_bytree=0.7, # The subsample ratio of rows as each tree is generated.Subsampling is done once per iteration.
                         subsample=0.7,  #default=1.the proportion regarded as training data. In this case, we use 70% as training data.
                         eta=0.3, 
                         seed=0)
xgb_model.fit(xgb_train, 
              Y_train, 
              eval_metric="rmse", 
              eval_set=[(xgb_train, Y_train), (xgb_val, Y_validation)], 
              verbose=20, 
              early_stopping_rounds=20)

In [64]:
#get feature importance and visualizing
from xgboost import plot_importance
plt.rcParams["figure.figsize"] = (15, 10)
plot_importance(xgb_model)
plt.show()

In [65]:
#predict the model
xgb_train_pred = xgb_model.predict(xgb_train)
xgb_val_pred = xgb_model.predict(xgb_val)
xgb_test_pred = xgb_model.predict(xgb_test)

In [66]:
#get the error:rmse
print('Train rmse:', np.sqrt(mean_squared_error(Y_train, xgb_train_pred)))
print('Validation rmse:', np.sqrt(mean_squared_error(Y_validation, xgb_val_pred)))

In [67]:
xgb_test_pred

**4.2 Random Forest Modelling**

In [68]:
from sklearn.ensemble import RandomForestRegressor
#select the useful feature in this model
rf_features = ['shop_id', 'item_id', 'item_cnt', 'transactions', 'year','item_cnt_mean', 'item_cnt_std', 'item_cnt_shifted1', 'shop_mean', 'item_mean', 'item_trend', 'mean_item_cnt']
rf_train = X_train[rf_features]
rf_val = X_validation[rf_features]
rf_test = X_test[rf_features]

In [69]:
#build and fit model
rf_model = RandomForestRegressor(n_estimators=50, max_depth=7, random_state=0, n_jobs=-1)
rf_model.fit(rf_train, Y_train)

In [70]:
#make predictions
rf_train_pred = rf_model.predict(rf_train)
rf_val_pred = rf_model.predict(rf_val)
rf_test_pred = rf_model.predict(rf_test)

In [71]:
#show the result and error
print('Train rmse:', np.sqrt(mean_squared_error(Y_train, rf_train_pred)))
print('Validation rmse:', np.sqrt(mean_squared_error(Y_validation, rf_val_pred)))

In [72]:
#get the correlation between the predictions using different models
plt.scatter(rf_val_pred, xgb_val_pred)

**4.3 Linear Regression**

In [78]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
#fit the model,replacing the time
X = train_set.drop('item_cnt_month', axis=1)
#dealing with missing data filling with 0
Lr_X = X.dropna(axis=0)
y = train_set['item_cnt_month'].dropna(axis=1)
#fit the model
lr.fit(Lr_X,y)

In [ ]:
#make the prediction with linear regression model
lr_prediction=lr.predict(X)

**4.4 LSTM Modelling**

In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from keras.models import Sequential
from keras.layers import LSTM, Dense, Activation
from keras import backend as K
from nested_lstm import NestedLSTMCell, NestedLSTM
#for imply the nested LSTM model, we need to insert a support document from:https://github.com/titu1994/Nested-LSTM/tree/5ad1939c18b9ce66b16fab0f48e0528e0c584876
from __future__ import absolute_import
import warnings

from keras import backend as K
from keras import activations
from keras import initializers
from keras import regularizers
from keras import constraints
from keras.engine import Layer
from keras.engine import InputSpec
from keras.legacy import interfaces
from keras.layers import RNN
from keras.layers.recurrent import _generate_dropout_mask
from keras.layers import LSTMCell, LSTM


class NestedLSTMCell(Layer):
    """Nested NestedLSTM Cell class.

    Derived from the paper [Nested LSTMs](https://arxiv.org/abs/1801.10308)
    Ref: [Tensorflow implementation](https://github.com/hannw/nlstm)

    # Arguments
        units: Positive integer, dimensionality of the output space.
        depth: Depth of nesting of the memory component.
        activation: Activation function to use
            (see [activations](../activations.md)).
            If you pass None, no activation is applied
            (ie. "linear" activation: `a(x) = x`).
        recurrent_activation: Activation function to use
            for the recurrent step
            (see [activations](../activations.md)).
        cell_activation: Activation function of the first cell gate.
            Note that in the paper only the first cell_activation is identity.
            (see [activations](../activations.md)).
        use_bias: Boolean, whether the layer uses a bias vector.
        kernel_initializer: Initializer for the `kernel` weights matrix,
            used for the linear transformation of the inputs
            (see [initializers](../initializers.md)).
        recurrent_initializer: Initializer for the `recurrent_kernel`
            weights matrix,
            used for the linear transformation of the recurrent state
            (see [initializers](../initializers.md)).
        bias_initializer: Initializer for the bias vector
            (see [initializers](../initializers.md)).
        unit_forget_bias: Boolean.
            If True, add 1 to the bias of the forget gate at initialization.
            Setting it to true will also force `bias_initializer="zeros"`.
            This is recommended in [Jozefowicz et al.](http://www.jmlr.org/proceedings/papers/v37/jozefowicz15.pdf)
        kernel_regularizer: Regularizer function applied to
            the `kernel` weights matrix
            (see [regularizer](../regularizers.md)).
        recurrent_regularizer: Regularizer function applied to
            the `recurrent_kernel` weights matrix
            (see [regularizer](../regularizers.md)).
        bias_regularizer: Regularizer function applied to the bias vector
            (see [regularizer](../regularizers.md)).
        kernel_constraint: Constraint function applied to
            the `kernel` weights matrix
            (see [constraints](../constraints.md)).
        recurrent_constraint: Constraint function applied to
            the `recurrent_kernel` weights matrix
            (see [constraints](../constraints.md)).
        bias_constraint: Constraint function applied to the bias vector
            (see [constraints](../constraints.md)).
        dropout: Float between 0 and 1.
            Fraction of the units to drop for
            the linear transformation of the inputs.
        recurrent_dropout: Float between 0 and 1.
            Fraction of the units to drop for
            the linear transformation of the recurrent state.
        implementation: Implementation mode, must be 2.
            Mode 1 will structure its operations as a larger number of
            smaller dot products and additions, whereas mode 2 will
            batch them into fewer, larger operations. These modes will
            have different performance profiles on different hardware and
            for different applications.
    """

    def __init__(self, units, depth,
                 activation='tanh',
                 recurrent_activation='sigmoid',
                 cell_activation='linear',
                 use_bias=True,
                 kernel_initializer='glorot_uniform',
                 recurrent_initializer='orthogonal',
                 bias_initializer='zeros',
                 unit_forget_bias=False,
                 kernel_regularizer=None,
                 recurrent_regularizer=None,
                 bias_regularizer=None,
                 kernel_constraint=None,
                 recurrent_constraint=None,
                 bias_constraint=None,
                 dropout=0.,
                 recurrent_dropout=0.,
                 implementation=2,
                 **kwargs):
        super(NestedLSTMCell, self).__init__(**kwargs)

        if depth < 1:
            raise ValueError("`depth` must be at least 1. For better performance, consider using depth > 1.")

        if implementation != 1:
            warnings.warn(
                "Nested LSTMs only supports implementation 2 for the moment. Defaulting to implementation = 2")
            implementation = 2

        self.units = units
        self.depth = depth
        self.activation = activations.get(activation)
        self.recurrent_activation = activations.get(recurrent_activation)
        self.cell_activation = activations.get(cell_activation)
        self.use_bias = use_bias

        self.kernel_initializer = initializers.get(kernel_initializer)
        self.recurrent_initializer = initializers.get(recurrent_initializer)
        self.bias_initializer = initializers.get(bias_initializer)
        self.unit_forget_bias = unit_forget_bias

        self.kernel_regularizer = regularizers.get(kernel_regularizer)
        self.recurrent_regularizer = regularizers.get(recurrent_regularizer)
        self.bias_regularizer = regularizers.get(bias_regularizer)

        self.kernel_constraint = constraints.get(kernel_constraint)
        self.recurrent_constraint = constraints.get(recurrent_constraint)
        self.bias_constraint = constraints.get(bias_constraint)

        self.dropout = min(1., max(0., dropout))
        self.recurrent_dropout = min(1., max(0., recurrent_dropout))
        self.implementation = implementation
        self.state_size = tuple([self.units] * (self.depth + 1))
        self._dropout_mask = None
        self._nested_recurrent_masks = None

    def build(self, input_shape):
        input_dim = input_shape[-1]
        self.kernels = []
        self.biases = []

        for i in range(self.depth):
            if i == 0:
                input_kernel = self.add_weight(shape=(input_dim, self.units * 4),
                                               name='input_kernel_%d' % (i + 1),
                                               initializer=self.kernel_initializer,
                                               regularizer=self.kernel_regularizer,
                                               constraint=self.kernel_constraint)
                hidden_kernel = self.add_weight(shape=(self.units, self.units * 4),
                                                name='kernel_%d' % (i + 1),
                                                initializer=self.recurrent_initializer,
                                                regularizer=self.recurrent_regularizer,
                                                constraint=self.recurrent_constraint)
                kernel = K.concatenate([input_kernel, hidden_kernel], axis=0)
            else:
                kernel = self.add_weight(shape=(self.units * 2, self.units * 4),
                                         name='kernel_%d' % (i + 1),
                                         initializer=self.recurrent_initializer,
                                         regularizer=self.recurrent_regularizer,
                                         constraint=self.recurrent_constraint)
            self.kernels.append(kernel)

        if self.use_bias:
            if self.unit_forget_bias:
                def bias_initializer(_, *args, **kwargs):
                    return K.concatenate([
                        self.bias_initializer((self.units,), *args, **kwargs),
                        initializers.Ones()((self.units,), *args, **kwargs),
                        self.bias_initializer((self.units * 2,), *args, **kwargs),
                    ])
            else:
                bias_initializer = self.bias_initializer

            for i in range(self.depth):
                bias = self.add_weight(shape=(self.units * 4,),
                                       name='bias_%d' % (i + 1),
                                       initializer=bias_initializer,
                                       regularizer=self.bias_regularizer,
                                       constraint=self.bias_constraint)
                self.biases.append(bias)
        else:
            self.biases = None

        self.built = True

    def call(self, inputs, states, training=None):
        if 0 < self.dropout < 1 and self._dropout_mask is None:
            self._dropout_mask = _generate_dropout_mask(
                K.ones_like(inputs),
                self.dropout,
                training=training,
                count=1)
        if (0 < self.recurrent_dropout < 1 and
                self._nested_recurrent_masks is None):
            _nested_recurrent_mask = _generate_dropout_mask(
                K.ones_like(states[0]),
                self.recurrent_dropout,
                training=training,
                count=self.depth)
            self._nested_recurrent_masks = _nested_recurrent_mask

        # dropout matrices for input units
        dp_mask = self._dropout_mask
        # dropout matrices for recurrent units
        rec_dp_masks = self._nested_recurrent_masks

        h_tm1 = states[0]  # previous memory state
        c_tm1 = states[1:self.depth + 1]  # previous carry states

        if 0. < self.dropout < 1.:
            inputs *= dp_mask[0]

        h, c = self.nested_recurrence(inputs,
                                      hidden_state=h_tm1,
                                      cell_states=c_tm1,
                                      recurrent_masks=rec_dp_masks,
                                      current_depth=0)

        if 0 < self.dropout + self.recurrent_dropout:
            if training is None:
                h._uses_learning_phase = True
        return h, c

    def nested_recurrence(self, inputs, hidden_state, cell_states, recurrent_masks, current_depth):
        h_state = hidden_state
        c_state = cell_states[current_depth]

        if 0.0 < self.recurrent_dropout <= 1. and recurrent_masks is not None:
            hidden_state = h_state * recurrent_masks[current_depth]

        ip = K.concatenate([inputs, hidden_state], axis=-1)
        gate_inputs = K.dot(ip, self.kernels[current_depth])

        if self.use_bias:
            gate_inputs = K.bias_add(gate_inputs, self.biases[current_depth])

        i = gate_inputs[:, :self.units]  # input gate
        f = gate_inputs[:, self.units * 2: self.units * 3]  # forget gate
        c = gate_inputs[:, self.units: 2 * self.units]  # new input
        o = gate_inputs[:, self.units * 3: self.units * 4]  # output gate

        inner_hidden = c_state * self.recurrent_activation(f)

        if current_depth == 0:
            inner_input = self.recurrent_activation(i) + self.cell_activation(c)
        else:
            inner_input = self.recurrent_activation(i) + self.activation(c)

        if (current_depth == self.depth - 1):
            new_c = inner_hidden + inner_input
            new_cs = [new_c]
        else:
            new_c, new_cs = self.nested_recurrence(inner_input,
                                                   hidden_state=inner_hidden,
                                                   cell_states=cell_states,
                                                   recurrent_masks=recurrent_masks,
                                                   current_depth=current_depth + 1)

        new_h = self.activation(new_c) * self.recurrent_activation(o)
        new_cs = [new_h] + new_cs

        return new_h, new_cs

    def get_config(self):
        config = {'units': self.units,
                  'depth': self.depth,
                  'activation': activations.serialize(self.activation),
                  'recurrent_activation': activations.serialize(self.recurrent_activation),
                  'cell_activation': activations.serialize(self.cell_activation),
                  'use_bias': self.use_bias,
                  'kernel_initializer': initializers.serialize(self.kernel_initializer),
                  'recurrent_initializer': initializers.serialize(self.recurrent_initializer),
                  'bias_initializer': initializers.serialize(self.bias_initializer),
                  'unit_forget_bias': self.unit_forget_bias,
                  'kernel_regularizer': regularizers.serialize(self.kernel_regularizer),
                  'recurrent_regularizer': regularizers.serialize(self.recurrent_regularizer),
                  'bias_regularizer': regularizers.serialize(self.bias_regularizer),
                  'kernel_constraint': constraints.serialize(self.kernel_constraint),
                  'recurrent_constraint': constraints.serialize(self.recurrent_constraint),
                  'bias_constraint': constraints.serialize(self.bias_constraint),
                  'dropout': self.dropout,
                  'recurrent_dropout': self.recurrent_dropout,
                  'implementation': self.implementation}
        base_config = super(NestedLSTMCell, self).get_config()
        return dict(list(base_config.items()) + list(config.items()))


class NestedLSTM(RNN):
    """Nested Long-Short-Term-Memory layer - [Nested LSTMs](https://arxiv.org/abs/1801.10308).

    # Arguments
        units: Positive integer, dimensionality of the output space.
        depth: Depth of nesting of the memory component.
        activation: Activation function to use
            (see [activations](../activations.md)).
            If you pass None, no activation is applied
            (ie. "linear" activation: `a(x) = x`).
        recurrent_activation: Activation function to use
            for the recurrent step
            (see [activations](../activations.md)).
        cell_activation: Activation function of the first cell gate.
            Note that in the paper only the first cell_activation is identity.
            (see [activations](../activations.md)).
        use_bias: Boolean, whether the layer uses a bias vector.
        kernel_initializer: Initializer for the `kernel` weights matrix,
            used for the linear transformation of the inputs.
            (see [initializers](../initializers.md)).
        recurrent_initializer: Initializer for the `recurrent_kernel`
            weights matrix,
            used for the linear transformation of the recurrent state.
            (see [initializers](../initializers.md)).
        bias_initializer: Initializer for the bias vector
            (see [initializers](../initializers.md)).
        unit_forget_bias: Boolean.
            If True, add 1 to the bias of the forget gate at initialization.
            Setting it to true will also force `bias_initializer="zeros"`.
            This is recommended in [Jozefowicz et al.](http://www.jmlr.org/proceedings/papers/v37/jozefowicz15.pdf)
        kernel_regularizer: Regularizer function applied to
            the `kernel` weights matrix
            (see [regularizer](../regularizers.md)).
        recurrent_regularizer: Regularizer function applied to
            the `recurrent_kernel` weights matrix
            (see [regularizer](../regularizers.md)).
        bias_regularizer: Regularizer function applied to the bias vector
            (see [regularizer](../regularizers.md)).
        activity_regularizer: Regularizer function applied to
            the output of the layer (its "activation").
            (see [regularizer](../regularizers.md)).
        kernel_constraint: Constraint function applied to
            the `kernel` weights matrix
            (see [constraints](../constraints.md)).
        recurrent_constraint: Constraint function applied to
            the `recurrent_kernel` weights matrix
            (see [constraints](../constraints.md)).
        bias_constraint: Constraint function applied to the bias vector
            (see [constraints](../constraints.md)).
        dropout: Float between 0 and 1.
            Fraction of the units to drop for
            the linear transformation of the inputs.
        recurrent_dropout: Float between 0 and 1.
            Fraction of the units to drop for
            the linear transformation of the recurrent state.
        implementation: Implementation mode, either 1 or 2.
            Mode 1 will structure its operations as a larger number of
            smaller dot products and additions, whereas mode 2 will
            batch them into fewer, larger operations. These modes will
            have different performance profiles on different hardware and
            for different applications.
        return_sequences: Boolean. Whether to return the last output.
            in the output sequence, or the full sequence.
        return_state: Boolean. Whether to return the last state
            in addition to the output.
        go_backwards: Boolean (default False).
            If True, process the input sequence backwards and return the
            reversed sequence.
        stateful: Boolean (default False). If True, the last state
            for each sample at index i in a batch will be used as initial
            state for the sample of index i in the following batch.
        unroll: Boolean (default False).
            If True, the network will be unrolled,
            else a symbolic loop will be used.
            Unrolling can speed-up a RNN,
            although it tends to be more memory-intensive.
            Unrolling is only suitable for short sequences.

    # References
        - [Long short-term memory](http://www.bioinf.jku.at/publications/older/2604.pdf) (original 1997 paper)
        - [Learning to forget: Continual prediction with NestedLSTM](http://www.mitpressjournals.org/doi/pdf/10.1162/089976600300015015)
        - [Supervised sequence labeling with recurrent neural networks](http://www.cs.toronto.edu/~graves/preprint.pdf)
        - [A Theoretically Grounded Application of Dropout in Recurrent Neural Networks](http://arxiv.org/abs/1512.05287)
        - [Nested LSTMs](https://arxiv.org/abs/1801.10308)
    """

    @interfaces.legacy_recurrent_support
    def __init__(self, units, depth,
                 activation='tanh',
                 recurrent_activation='sigmoid',
                 cell_activation='linear',
                 use_bias=True,
                 kernel_initializer='glorot_uniform',
                 recurrent_initializer='orthogonal',
                 bias_initializer='zeros',
                 unit_forget_bias=False,
                 kernel_regularizer=None,
                 recurrent_regularizer=None,
                 bias_regularizer=None,
                 activity_regularizer=None,
                 kernel_constraint=None,
                 recurrent_constraint=None,
                 bias_constraint=None,
                 dropout=0.,
                 recurrent_dropout=0.,
                 implementation=1,
                 return_sequences=False,
                 return_state=False,
                 go_backwards=False,
                 stateful=False,
                 unroll=False,
                 **kwargs):
        if implementation == 0:
            warnings.warn('`implementation=0` has been deprecated, '
                          'and now defaults to `implementation=2`.'
                          'Please update your layer call.')
        if K.backend() == 'theano':
            warnings.warn(
                'RNN dropout is no longer supported with the Theano backend '
                'due to technical limitations. '
                'You can either set `dropout` and `recurrent_dropout` to 0, '
                'or use the TensorFlow backend.')
            dropout = 0.
            recurrent_dropout = 0.

        cell = NestedLSTMCell(units, depth,
                              activation=activation,
                              recurrent_activation=recurrent_activation,
                              cell_activation=cell_activation,
                              use_bias=use_bias,
                              kernel_initializer=kernel_initializer,
                              recurrent_initializer=recurrent_initializer,
                              unit_forget_bias=unit_forget_bias,
                              bias_initializer=bias_initializer,
                              kernel_regularizer=kernel_regularizer,
                              recurrent_regularizer=recurrent_regularizer,
                              bias_regularizer=bias_regularizer,
                              kernel_constraint=kernel_constraint,
                              recurrent_constraint=recurrent_constraint,
                              bias_constraint=bias_constraint,
                              dropout=dropout,
                              recurrent_dropout=recurrent_dropout,
                              implementation=implementation)
        super(NestedLSTM, self).__init__(cell,
                                         return_sequences=return_sequences,
                                         return_state=return_state,
                                         go_backwards=go_backwards,
                                         stateful=stateful,
                                         unroll=unroll,
                                         **kwargs)
        self.activity_regularizer = regularizers.get(activity_regularizer)

    def call(self, inputs, mask=None, training=None, initial_state=None, constants=None):
        self.cell._dropout_mask = None
        self.cell._nested_recurrent_masks = None
        return super(NestedLSTM, self).call(inputs,
                                            mask=mask,
                                            training=training,
                                            initial_state=initial_state,
                                            constants=constants)

    @property
    def units(self):
        return self.cell.units

    @property
    def depth(self):
        return self.cell.depth

    @property
    def activation(self):
        return self.cell.activation

    @property
    def recurrent_activation(self):
        return self.cell.recurrent_activation

    @property
    def cell_activation(self):
        return self.cell.cell_activation

    @property
    def use_bias(self):
        return self.cell.use_bias

    @property
    def kernel_initializer(self):
        return self.cell.kernel_initializer

    @property
    def recurrent_initializer(self):
        return self.cell.recurrent_initializer

    @property
    def bias_initializer(self):
        return self.cell.bias_initializer

    @property
    def unit_forget_bias(self):
        return self.cell.unit_forget_bias

    @property
    def kernel_regularizer(self):
        return self.cell.kernel_regularizer

    @property
    def recurrent_regularizer(self):
        return self.cell.recurrent_regularizer

    @property
    def bias_regularizer(self):
        return self.cell.bias_regularizer

    @property
    def kernel_constraint(self):
        return self.cell.kernel_constraint

    @property
    def recurrent_constraint(self):
        return self.cell.recurrent_constraint

    @property
    def bias_constraint(self):
        return self.cell.bias_constraint

    @property
    def dropout(self):
        return self.cell.dropout

    @property
    def recurrent_dropout(self):
        return self.cell.recurrent_dropout

    @property
    def implementation(self):
        return self.cell.implementation

    def get_config(self):
        config = {'units': self.units,
                  'depth': self.depth,
                  'activation': activations.serialize(self.activation),
                  'recurrent_activation': activations.serialize(self.recurrent_activation),
                  'cell_activation': activations.serialize(self.cell_activation),
                  'use_bias': self.use_bias,
                  'kernel_initializer': initializers.serialize(self.kernel_initializer),
                  'recurrent_initializer': initializers.serialize(self.recurrent_initializer),
                  'bias_initializer': initializers.serialize(self.bias_initializer),
                  'unit_forget_bias': self.unit_forget_bias,
                  'kernel_regularizer': regularizers.serialize(self.kernel_regularizer),
                  'recurrent_regularizer': regularizers.serialize(self.recurrent_regularizer),
                  'bias_regularizer': regularizers.serialize(self.bias_regularizer),
                  'activity_regularizer': regularizers.serialize(self.activity_regularizer),
                  'kernel_constraint': constraints.serialize(self.kernel_constraint),
                  'recurrent_constraint': constraints.serialize(self.recurrent_constraint),
                  'bias_constraint': constraints.serialize(self.bias_constraint),
                  'dropout': self.dropout,
                  'recurrent_dropout': self.recurrent_dropout,
                  'implementation': self.implementation}
        base_config = super(NestedLSTM, self).get_config()
        del base_config['cell']
        return dict(list(base_config.items()) + list(config.items()))

    @classmethod
    def from_config(cls, config):
        if 'implementation' in config and config['implementation'] == 0:
            config['implementation'] = 2
        return cls(**config)

In [ ]:
from IPython.display import Image
Image("../input/nested-lstm/2022-07-07 8.59.16.png")

In [ ]:
from IPython.display import Image
Image("../input/nested-lstm/2022-07-07 8.59.06.png")

In [ ]:
#establish the model
LSTM = Sequential()
LSTM.add(NestedLSTM(40, input_shape=(33, 1), depth=2, dropout=0.0, recurrent_dropout=0.0))
LSTM.add(Dense(1))
# use adam optimizer 
LSTM.compile(loss='mse',optimizer='adam',metrics=['mean_squared_error'])
LSTM.summary()

In [ ]:
#generate a dataframe df,deal with datatime with year-month format
LSTM_df = sales.groupby([sales.date.apply(lambda x: x.strftime('%Y-%m')),'item_id','shop_id']).sum().reset_index()

In [ ]:
#table the datatime(year-month)
LSTM_df = df[['date','item_id','shop_id','item_cnt_day']]
LSTM_df = LSTM_df.pivot_table(index=['item_id','shop_id'], columns='date',values='item_cnt_day',fill_value=0).reset_index()
LSTM_df.head()

In [ ]:
#generate the test dataset with merging method
LSTM_df_test = pd.merge(test,LSTM_df, on=['item_id','shop_id'], how='left')
LSTM_df_test = df_test.fillna(0) #dealing with missing data filling with 0
LSTM_df_test.head()

In [ ]:
LSTM_df_test = LSTM_df_test.drop(labels=['ID', 'shop_id', 'item_id'], axis=1)
LSTM_df_test.head()

In [ ]:
#generate the train dataset,define the input sequence
TARGET = '2015-10' #set up the target data
LSTM_y_train = LSTM_df_test[TARGET]
LSTM_X_train = LSTM_df_test.drop(labels=[TARGET], axis=1)
LSTM_X_train = LSTM_X_train.as_matrix() 
#reshape the imput according to (samples, timesteps, features)
LSTM_X_train = LSTM_X_train.reshape((214200, 33, 1)) 

LSTM_y_train = LSTM_y_train.as_matrix()
LSTM_y_train = LSTM_y_train.reshape(214200, 1) # Timestep dimension will be squeezed in the target, when the target has to be reshaped to match the same pattern

LSTM_X_test = LSTM_df_test.drop(labels=['2013-01'],axis=1)
LSTM_X_test = LSTM_X_test.as_matrix()
LSTM_X_test = LSTM_X_test.reshape((214200, 33, 1))

LSTM_df_test = pd.merge(LSTM_df_test, LSTM_df, on=['item_id','shop_id'], how='left') #merge the test and lstm dataframe together 
LSTM_df_test = LSTM_df_test.fillna(0) #deal with missing value and fill them with zero
LSTM_df_test.head()

In [ ]:
#fit the model,reduce batch size and increase number of epochs, see where it converges.
BATCH = 32
LSTM.fit(LSTM_X_train, LSTM_y_train,
          batch_size=BATCH,
          epochs=50)
#Prediction
prediction=LSTM.predict(LSTM_df_test,verbose=0)